In [72]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup

from selenium.common.exceptions import NoSuchElementException

from PIL import Image
import io

import time

import pandas as pd

**ブラウザの起動**

**各ページのURLを取得する関数**
1. SUUMOの物件情報(indexページ)は、基本的に内部リンク（'https:~がない"）形式でHTMLが作成されているため、内部リンクのみ全体のページから取得する
2. 取得したURLをリスト形式にまとめる
3. URLのリストを返す関数を作成する

In [2]:
def get_urls(soup) -> list:
    """
    一覧のページからURL個別ページのURLを取得する関数
    """
    h2_elems = soup.find_all('h2', attrs={'class' : 'property_unit-title'})
    
    urls = []
    
    for h2_elem in h2_elems:
        a_elem = h2_elem.find('a')
        url = a_elem.attrs['href']
        
        urls.append(url)
        
    return urls

**各ページの物件情報を収集する関数**
- 入力は各ページの内部リンクのURL（シングル）
- 出力は各ページのsoup

In [3]:
def get_page_soup(internal_url:str):
    """
    各物件へのリンクを作成して、アクセスして、各ページのsoupを出力する
    """
    # ページ内リンクをドメインと結びつけて直にアクセスできるリンクに変換している
    page_url = 'https://suumo.jp' + internal_url
    page_res = requests.get(page_url)
    page_soup = BeautifulSoup(page_res.content, 'html.parser')
    
    return page_soup

**各ページの物件情報を収集する関数**
- 入力は各ページのsoup
- 出力は各ページの物件情報が格納された辞書

In [4]:
def get_house_details(page_soup) -> dict:
    """[summary]　
    　その後に各物件のページから「物件情報」にアクセスして、「物件情報」のテーブルまるごと取得する。
    Args
        internal_url[str]:SUUMOの各物件の内部リンク
    """
    #物件詳細のページへのリンクの取得
    house_details_a_elem = page_soup.find('a', attrs={'class' : 'tabOutline2'})
    
    if house_details_a_elem is None:
        house_details_a_elem = page_soup.find('a', attrs={'class' : 'tabOutline'})
        
    house_details_url = house_details_a_elem.attrs['href']
    
    #物件詳細のページへのアクセス    
    house_details_res = requests.get(house_details_url)
    house_details_soup = BeautifulSoup(house_details_res.content, 'html.parser')
    
    # テーブルの取得
    house_details_info = house_details_soup.find('table', {'class': 'pCell10'})

    return house_details_info

**各ページの物件情報を収集する関数**
- 入力はtable要素
- 出力はth要素がKey、td要素がValueになった辞書

In [5]:
def extract_table_data(table) -> dict:
    """
    　tableのHTMLからthとtd内の情報を取得する　
    """
    house_details_dict = {}

    for row in table.find_all('tr'):
        data_head = row.find_all('th')
        data_value = row.find_all('td')

        data_head_01 = data_head[0].text.replace('\n', '').replace('ヒント', '')
        data_value_01 = data_value[0].text.replace('\n', '').replace('\r', '').replace('\t', '')

        try:
            if data_head[1] is not None:
                data_head_02 = data_head[1].text.replace('\n', '').replace('ヒント', '')
                data_value_02 = data_value[1].text.replace('\n', '').replace('\r', '').replace('\t', '').replace('\u3000', '').replace('\xa0', '').replace('[乗り換え案内]', '')

                house_details_dict[data_head_01] = data_value_01
                house_details_dict[data_head_02] = data_value_02
        except IndexError:
            house_details_dict[data_head_01] = data_value_01
            
    return house_details_dict

**各物件のタイトルとコメントを取得する関数**

In [6]:
def get_title_and_comment(page_soup):
    """
    サイト上部にあるタイトルとコメントを取得する関数
    """
    try:
        house_title = page_soup.find('h2', attrs = {'class' : 'fs16'})
        house_comment = page_soup.find('p', attrs = {'class' : 'fs14'})
        house_dict = {'title':house_title.text, 'comment':house_comment.text}
    except:
        house_dict = {'title':'', 'comment':''}
    return house_dict

## テスト用のコードTips

In [92]:
# page_soup = get_page_soup(urls[1]) 
# print(type(page_soup))
# table = get_house_details(page_soup)
# print(type(table))
# house_info_dict = extract_table_data(table)
# house_text_dict = get_title_and_comment(page_soup)
# get_house_img(page_soup, 1)
# urls

<class 'bs4.BeautifulSoup'>
<class 'bs4.element.Tag'>
1


['/chukoikkodate/yamagata/sc_tendo/nc_97027597/',
 '/chukoikkodate/yamagata/sc_yamagata/nc_96589986/',
 '/chukoikkodate/yamagata/sc_yamagata/nc_97426196/',
 '/chukoikkodate/yamagata/sc_yamagata/nc_96140671/',
 '/chukoikkodate/yamagata/sc_sakata/nc_97073131/',
 '/chukoikkodate/yamagata/sc_higashimurayamagun/nc_96627907/',
 '/chukoikkodate/yamagata/sc_yamagata/nc_95997891/',
 '/chukoikkodate/yamagata/sc_higashimurayamagun/nc_97362727/',
 '/chukoikkodate/yamagata/sc_yamagata/nc_92020328/',
 '/chukoikkodate/yamagata/sc_yamagata/nc_95803869/',
 '/chukoikkodate/yamagata/sc_tendo/nc_94966659/',
 '/chukoikkodate/yamagata/sc_sakata/nc_94599713/',
 '/chukoikkodate/yamagata/sc_yamagata/nc_96872001/']

**家の画像の取得**
- 画像の名前を一意に決めるためにhouse_idを引数に入れている

In [79]:
def get_house_img(page_soup, house_id):
    """
    各ページの写真を取得して、写真をHouseId_IMGIDの形式で保存する。
    """
    print(house_id)
    imgs = page_soup.find_all('img')
    img_list = list()
    img_id = 0
    # 以下で始まるのはIMGタグだがアクセスできないため排除する
    un_img_signal = 'gvavadfbasdfbarvbaebabaertbertbaebfbadbavafdvkavnakfvbaklvbaiklvuhiaerbnvnvkajbvkajbfgkjasbvkabvabfoak;dnvlasndvkahgvklashdvb'
    
    for img in imgs:
        try:
            #print('img', img)
            img_url = img['rel']
            img_tag = img['alt']
        except KeyError:
            img_url = un_img_signal
            img_tag = ""

        if not un_img_signal in img_url:
            #print("house_id", house_id, "img_id", img_id, 'img_url', str(img_url))
            img_name = str(house_id) + '_' + str(img_id)
            img_id += 1
            img_url = img_url.replace('&amp;', '&')
            img = Image.open(io.BytesIO(requests.get(img_url).content))
            img.save(f'imgs/{img_name}.jpg')
            img_list.append({'house_id':house_id, 'img_id':img_id, 'img_tag':img_tag, 'img_name':img_name})
    
    return img_list

**Index1ページ分のURL**

- ここのループでは、Index1ページ分のURLをすべて取ってきている

In [9]:
def get_index_info(urls:list, house_info:list, house_id:int) -> list:
    for url in urls:
        page_soup = get_page_soup(url)
        table = get_house_details(page_soup)
        house_info_dict = extract_table_data(table)
        house_text_dict = get_title_and_comment(page_soup)
        house_img_list = get_house_img(page_soup, house_id)

        house_dict = {'House_ID': house_id, 'text':house_text_dict, 'info':house_info_dict, 'imgs':house_img_list}
        house_info.append(house_dict)
        
        house_id += 1
        
        time.sleep(20)
        
    return house_info, house_id

In [10]:
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
url = 'https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=020&bs=021&ta=06&jspIdFlg=patternShikugun&kb=1&kt=9999999&tb=0&tt=9999999&hb=0&ht=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999'
browser.get(url)

count = 0

url = browser.current_url
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
a_elems = soup.find_all('div', attrs={'class' : 'property_unit-header'})



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


**すべてのページを見れるように次へのボタンを押す**

- このままでは、すべてのページを自動的にクロールすることは不可能。そこで、すべてのページをクロールできるようにプログラムで次へのボタンを押すことを試みます。

In [80]:
data_list = []
#house_info = []
house_id = 0

count = 0

url = 'https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=020&bs=021&ta=06&jspIdFlg=patternShikugun&kb=1&kt=9999999&tb=0&tt=9999999&hb=0&ht=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999'
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
browser.get(url)

while True:
    url = browser.current_url
    print(url)
    #ブラウザの起動
    options = Options()
    options.add_argument('--headless')
    browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    
    browser.get(url)
    
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'html.parser')
        #a_elems = soup.find_all('div', attrs={'class' : 'property_unit-header'})
        urls = get_urls(soup)
        house_info, house_id = get_index_info(urls, data_list, house_id)
        
        browser.find_element_by_link_text('次へ').click()
        if count % 10 == 0:
            print("pages:{0}".format(count))
            print('==============================================')
            time.sleep(60)
        time.sleep(60)
        
        count += 1
            
    except NoSuchElementException:
        browser.quit()
        break

print('browser quit')



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=020&bs=021&ta=06&jspIdFlg=patternShikugun&kb=1&kt=9999999&tb=0&tt=9999999&hb=0&ht=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
pages:0




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=020&bs=021&ta=06&jspIdFlg=patternShikugun&kb=1&kt=9999999&tb=0&tt=9999999&hb=0&ht=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999&pn=2
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


...

https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=020&bs=021&ta=06&jspIdFlg=patternShikugun&kb=1&kt=9999999&tb=0&tt=9999999&hb=0&ht=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999&pn=10
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=020&bs=021&ta=06&jspIdFlg=patternShikugun&kb=1&kt=9999999&tb=0&tt=9999999&hb=0&ht=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999&pn=11
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
pages:10




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=020&bs=021&ta=06&jspIdFlg=patternShikugun&kb=1&kt=9999999&tb=0&tt=9999999&hb=0&ht=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999&pn=12
330
331
332
333
334
335
336
337
338
339
340
341
342
browser quit


In [78]:
house_info

[{'House_ID': 0,
  'text': {'title': '令和２年２月末内装リフォーム済み！閑静な住宅地！国道112号線へのアクセス良し！',
   'comment': '【リフォーム内容】ユニットバス交換・換気扇取替・トイレ交換・CF貼替・キッチンシンク交換・洗面化粧台交換・クロス貼替・床一部張替・建具調整・畳表替・障子張替・襖張替・照明器具交換・ハウスクリーニング駐車２台可、即入居可、内装リフォーム、南向き、陽当り良好、角地、和室'},
  'info': {'販売スケジュール': '-',
   'イベント情報': '-',
   '所在地': '山形県東村山郡中山町大字長崎',
   '交通': 'ＪＲ左沢線「羽前長崎」歩22分',
   '販売戸数': '1戸',
   '総戸数': '1戸',
   '価格': '1098万円[ □支払シミュレーション ]',
   '最多価格帯': '-',
   '私道負担・道路': '無、西6ｍ幅（接道幅9ｍ）、北6ｍ幅（接道幅9ｍ）',
   '諸費用': '-',
   '間取り': '4LDK',
   '建物面積': '84.84m2（登記）',
   '土地面積': '164.02m2（登記）',
   '建ぺい率・容積率': '60％・200％',
   '完成時期(築年月)': '1985年6月',
   '入居時期': '即入居可',
   '土地の権利形態': '所有権',
   '構造・工法': '木造2階建',
   '施工': '-',
   'リフォーム': '2020年2月完了水回り設備交換：キッチン・浴室・トイレ※年月は一番古いリフォーム箇所を表します',
   '用途地域': '１種住居',
   '地目': '-',
   'その他制限事項': '-',
   'その他概要・特記事項': '設備：公営水道、個別浄化槽、個別LPG、駐車場：カースペース'},
  'imgs': [{'house_id': 0,
    'img_id': 1,
    'img_tag': '現地外観写真',
    'img_name': '0_0'},
   {'house_id': 0, 'img_id': 2, 'img_tag': '間取り図', 'img_name':

In [58]:
len(house_info)

335

In [86]:
def edit_house_data(house):
    house_id = house['House_ID']
    house_dict = {
        'title' : house['text']['title'],
        'comment' : house['text']['comment'],
        '販売スケジュール' : house['info']['販売スケジュール'],
        'イベント情報' : house['info']['イベント情報'],
        '所在地' : house['info']['所在地'],
        '交通' : house['info']['交通'],
        '販売戸数' : house['info']['販売戸数'],
        '総戸数' : house['info']['総戸数'],
        '価格' : house['info']['価格'],
        '最多価格帯' : house['info']['最多価格帯'],
        '私道負担・道路' : house['info']['私道負担・道路'],
        '諸費用' : house['info']['諸費用'],
        '間取り' : house['info']['間取り'],
        '建物面積' : house['info']['建物面積'],
        '土地面積' : house['info']['土地面積'],
        '建ぺい率・容積率' : house['info']['建ぺい率・容積率'],
        '完成時期(築年月)' : house['info']['完成時期(築年月)'],
        '入居時期' : house['info']['入居時期'],
        '土地の権利形態' : house['info']['土地の権利形態'],
        '構造・工法' : house['info']['構造・工法'],
        '施工' : house['info']['施工'],
        'リフォーム' : house['info']['リフォーム'],
        '用途地域' : house['info']['用途地域'],
        '地目' : house['info']['地目'],
        'その他制限事項' : house['info']['その他制限事項'],
        'その他概要・特記事項' : house['info']['その他概要・特記事項'],
    }
    
    return house_id, house_dict

In [87]:
houses_dict = {}
img_list = []

for house in house_info:
    # Pandasで加工しやすいようにKeyが一つの辞書に家の情報を変更する
    house_id, house_dict = edit_house_data(house)
    houses_dict[house_id] = house_dict
    # Houseの画像は変更が必要ないので、そのままリストに追加する
    # ただ、一つづつ取り出して追加する必要があるので、要注意
    img_list += house['imgs']

In [88]:

#houses_dict
attribute_df = pd.DataFrame(houses_dict).transpose()
attribute_df.to_csv('csv/attribute_{filename}.csv'.format(filename = "Yamagata"))
imgs_df = pd.DataFrame(img_list)
imgs_df.to_csv('csv/imgs_{filename}.csv'.format(filename = "Yamagata"))